In [1]:
api_key = "9c461c13ec4b4491b4d5905948bb7805313adca82287ca6997405972c026a0e5"
api_secret = "cd8eb32222b99586ca8967eb7325b325f5ab0a4f4d2aa536bdd215aa01801ea5"

In [4]:
from binance.client import Client
import numpy as np
import pandas as pd
from binance import ThreadedWebsocketManager
import pandas as pd
import numpy as np
from datetime import datetime,  timedelta
import time

client = Client(api_key = api_key, api_secret = api_secret, tld = "com", testnet = True)

In [ ]:
from datetime import date
import symbol


class Futurestrader():
    def __init__(self, symbol, bar_len, sma_s, sma_m, sma_l, units ,position = 0, leverage = 5):
        self.symbol = symbol
        self.bar_len = bar_len
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        self.units = units
        self.postion = position
        self.leverage = leverage
        self.cum_profit = 0

        self.sma_s = sma_s
        self.sma_m = sma_m
        self.sma_l = sma_l
        
    def start_trader(self, historical_days):
        client.futures_change_leverage(symbol = "BTCUSDT", leverage = self.leverage)
        self.twm = ThreadedWebsocketManager
        self.twm.start()

        if self.bar_len in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_len, days = historical_days)
            self.twm.start_kline_futures_socket(callback=self.stream_candles, symbol = self.symbol, interval = self.bar_len)

    def get_most_recent(self, symbol, interval, days):
    
        now = datetime.utcnow()
        past = str(now - timedelta(days = days))
    
        bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df
    
    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # print out
        print(".", end = "", flush = True) # just print something to get a feedback (everything OK) 
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
        # prepare features and define strategy/trading positions whenever the latest bar is complete
        if complete == True:
            self.define_strategy()
            self.execute_trades() # NEW!!!
        
    def define_strategy(self):
        
        df = self.data.copy()
        
        #******************** define your strategy here ************************
        df = df[["Close", "Volume"]].copy()
        df["returns"] = np.log(df.Close / df.Close.shift())
        df["vol_ch"] = np.log(df.Volume.div(df.Volume.shift(1)))
        df.loc[df.vol_ch > 3, "vol_ch"] = np.nan
        df.loc[df.vol_ch < -3, "vol_ch"] = np.nan  
        
        cond1 = df.returns >= self.return_thresh
        cond2 = df.vol_ch.between(self.volume_thresh[0], self.volume_thresh[1])
        
        df["position"] = 1
        df.loc[cond1 & cond2, "position"] = 0
        #***********************************************************************
        
        self.prepared_data = df.copy()
    
    def execute_trades(self): # NEW!
        if self.prepared_data["position"].iloc[-1] == 1: # if position is long -> go/stay long
            if self.position == 0:
                order = client.create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                print("GOING LONG")
            self.position = 1
        elif self.prepared_data["position"].iloc[-1] == 0: # if position is neutral -> go/stay neutral
            if self.position == 1:
                order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                print("GOING NEUTRAL")
            self.position = 0
